# Todoist Completed Tasks Downloader

This project will collect and aggregate all of your completed task data from Todoist. 

For a simple data analysis of your completed tasks, see [todoist_data_analysis.ipynb](https://github.com/markwk/qs_ledger/blob/master/todoist/todoist_data_analysis.ipynb). 

-------

## Installation and Setup

#### Download and Install Todoist Python Library

`$ pip install todoist-python`

#### Signup and Create a Todoist App

* Signup at https://developer.todoist.com/appconsole.html
* Once app is created, generate and copy a "Test token," which provides access to API as your user.
* Copy sample-credentials.json and create credentials.json
* Add and Save your Test Token to credentials.json


-----

## Dependencies

In [1]:
from todoist.api import TodoistAPI
import numpy as np, string, re, pytz
import pandas as pd
from datetime import datetime

### Credentials and Authentification

In [2]:
import json

with open("credentials.json", "r") as file:
    credentials = json.load(file)
    todoist_cr = credentials['todoist']
    TOKEN = todoist_cr['TOKEN']

In [3]:
api = TodoistAPI(TOKEN)
#api.sync() # uncomment to use

----------

## Check Basic User Info

In [4]:
user = api.state['user']

In [5]:
# user

In [6]:
user['full_name']

'Mark Koester'

In [7]:
# Tasks Completed Today
user['completed_today']

1

In [8]:
# total completed tasks
user_completed_count = user['completed_count']
user_completed_count

4698

-------

# List and Export of Current Projects

### API Call: api.state['projects']

https://developer.todoist.com/sync/v7/#get-all-projects

NOTE: This only gets info on your existing projects and exludes archived projects. 

In [9]:
user_projects  = api.state['projects']

In [10]:
# user_projects

In [11]:
len(user_projects)

41

In [12]:
with open('data/todoist-projects.csv', 'w') as file:
    file.write("Id" + "," + "Project" + "\n")
    for i in range(0, len(user_projects)):
        file.write('\"' + str(user_projects[i]['id']) + '\"' + "," + '\"' + str(user_projects[i]['name']) + '\"' + "\n")

In [13]:
projects = pd.read_csv("data/todoist-projects.csv")

In [14]:
# projects

-----

## User Completed Tasks Stats Info

API Call: `api.completed.get_stats()` https://developer.todoist.com/sync/v7/#get-productivity-stats

In [15]:
stats = api.completed.get_stats()

In [16]:
# total completed tasks from stats
user_completed_stats = stats['completed_count']
user_completed_stats

4988

-------

# Collect Raw List of All Completed Items from Todoist

### API Call: api.completed.get_all() 

https://developer.todoist.com/sync/v7/#get-all-completed-items

In [17]:
def get_completed_todoist_items():
    # create df from initial 50 completed tasks
    print("Collecting Initial 50 Completed Todoist Tasks...")
    temp_tasks_dict = (api.completed.get_all(limit=50))
    past_tasks = pd.DataFrame.from_dict(temp_tasks_dict['items'])
    # get the remaining items
    pager = list(range(50,user_completed_stats,50))
    for count, item in enumerate(pager):
        tmp_tasks = (api.completed.get_all(limit=50, offset=item))
        tmp_tasks_df = pd.DataFrame.from_dict(tmp_tasks['items'])
        past_tasks = pd.concat([past_tasks, tmp_tasks_df])
        print("Collecting Additional Todoist Tasks " + str(item) + " of " + str(user_completed_stats))
    # save to CSV
    print("...Generating CSV Export")
    past_tasks.to_csv("data/todost-raw-tasks-completed.csv", index=False)

In [18]:
get_completed_todoist_items()

...Generating CSV Export


In [19]:
past_tasks = pd.read_csv("data/todost-raw-tasks-completed.csv")

In [20]:
past_tasks.head()

,completed_date,content,id,meta_data,project_id,task_id,user_id
0,Thu 06 Dec 2018 10:19:02 +0000,READINGS: Running Stride and Cadence,2890761399,NaN,2158267779,2890761399,4288657
1,Thu 06 Dec 2018 10:19:02 +0000,Readings and Review on Running Stride,2890761573,NaN,2158267779,2890761573,4288657
2,Thu 06 Dec 2018 10:18:59 +0000,READ / Notes on Are Emotions of Natural Kind?,2939017612,NaN,178797715,2939017612,4288657
3,Thu 06 Dec 2018 10:18:57 +0000,Debug Site Down for GayRomLit,2939016120,NaN,179268527,2939016120,4288657
4,Wed 05 Dec 2018 16:45:33 +0000,Call with Jo Carol,2937620714,NaN,2182684133,2937620714,4288657


In [21]:
# generated count 
collected_total = len(past_tasks)
collected_total

4988

In [22]:
# Does our collected total tasks match stat of completed count on user
collected_total == user_completed_count

False

In [23]:
len(past_tasks.drop_duplicates())

4988

In [24]:
past_tasks['project_id'] = past_tasks.project_id.astype('category')

In [25]:
past_tasks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4988 entries, 0 to 4987
Data columns (total 7 columns):
completed_date    4988 non-null object
content           4988 non-null object
id                4988 non-null int64
meta_data         0 non-null float64
project_id        4988 non-null category
task_id           4988 non-null int64
user_id           4988 non-null int64
dtypes: category(1), float64(1), int64(3), object(2)
memory usage: 240.3+ KB


In [26]:
len(past_tasks.project_id.unique())

43

---------

## Get All Current and Previous Projects

In [27]:
# Extract all project ids used on tasks
project_ids = past_tasks.project_id.unique()
# project_ids

In [28]:
# total all-time projects
len(project_ids)

43

In [29]:
# get project info from Todoist API
def get_todoist_project_name(project_id):
    item = api.projects.get_by_id(project_id)
    if item: 
        try:
            return item['name']
        except:
            return item['project']['name']

In [30]:
# Testing with a Sample Archived Project
# get_todoist_project_name(183682060)

In [31]:
# Testing with a Sample Current Project
# get_todoist_project_name(1252539618)

In [32]:
# Get Info on All User Projects
project_names = []
for i in project_ids:
    project_names.append(get_todoist_project_name(i))

In [33]:
project_names

['Exercise',
 'Studies: General',
 'Misc Client Work',
 'BookLoversCon',
 'Health',
 'Int3 Biz Dev',
 'Writing',
 'Data-Driven You',
 'Tech / Computer',
 'Code Studies',
 'Productivity, Self-Tracking',
 'Podcast Tracker',
 'Financial / Investment',
 'Maintenance, Org, Cleaning',
 'Language Learning',
 'Networking / Career',
 'Goals',
 'Traveling',
 'Data',
 'Math',
 'Int3c Biz, Drupal Dev',
 'Design',
 'PhotoStats',
 'Video',
 'RTConvention',
 'Nutrition & Food',
 'RTBookReviews',
 'Drupal Studies & Dev',
 'Biomarker Tracker',
 'Entrepreneurship / Biz',
 'Creative',
 'Side Projects / Startups',
 'Philosophy',
 None,
 'Maker Building & Studies',
 'Exercise, Health, Fitness Related',
 'TMW/Nissan',
 'Personal',
 None,
 'Poetry',
 'Inbox',
 None,
 'RT']

-----

## Match Project Id Name on Completed Tasks, Add Day of Week

In [34]:
past_tasks.tail()

,completed_date,content,id,meta_data,project_id,task_id,user_id
4983,Sun 28 Aug 2016 12:07:13 +0000,Read Checklist from MASTER THE GAME,53281331,NaN,178797715,53281331,4288657
4984,Sun 28 Aug 2016 10:40:51 +0000,Weekly Review,53277061,NaN,142200795,53277061,4288657
4985,Sun 28 Aug 2016 10:24:01 +0000,Financial Reflection Writing,53275224,NaN,142200795,53275224,4288657
4986,Sun 28 Aug 2016 10:17:51 +0000,Study Todoist Shortcuts,53273289,NaN,142200795,53273289,4288657
4987,Sun 28 Aug 2016 06:39:21 +0000,Review & Setup Tasks on TODOIST,53265021,NaN,142200795,53265021,4288657


In [35]:
# Probably a more effecient way to do this
project_lookup = lambda x: get_todoist_project_name(x)

In [36]:
past_tasks['project_name'] = past_tasks['project_id'].apply(project_lookup) # note: not very efficient

In [37]:
len(past_tasks.project_name.unique())

41

In [38]:
# functions to convert UTC to Shanghai time zone and extract date/time elements
convert_tz = lambda x: x.to_pydatetime().replace(tzinfo=pytz.utc).astimezone(pytz.timezone('Asia/Shanghai'))
get_year = lambda x: convert_tz(x).year
get_month = lambda x: '{}-{:02}'.format(convert_tz(x).year, convert_tz(x).month) #inefficient
get_date = lambda x: '{}-{:02}-{:02}'.format(convert_tz(x).year, convert_tz(x).month, convert_tz(x).day) #inefficient
get_day = lambda x: convert_tz(x).day
get_hour = lambda x: convert_tz(x).hour
get_day_of_week = lambda x: convert_tz(x).weekday()

In [39]:
# parse out date and time elements as Shanghai time
past_tasks['completed_date'] = pd.to_datetime(past_tasks['completed_date'])
past_tasks['year'] = past_tasks['completed_date'].map(get_year)
past_tasks['month'] = past_tasks['completed_date'].map(get_month)
past_tasks['date'] = past_tasks['completed_date'].map(get_date)
past_tasks['day'] = past_tasks['completed_date'].map(get_day)
past_tasks['hour'] = past_tasks['completed_date'].map(get_hour)
past_tasks['dow'] = past_tasks['completed_date'].map(get_day_of_week)
past_tasks = past_tasks.drop(labels=['completed_date'], axis=1)

In [40]:
past_tasks.head()

,content,id,meta_data,project_id,task_id,user_id,project_name,year,month,date,day,hour,dow
0,READINGS: Running Stride and Cadence,2890761399,NaN,2158267779,2890761399,4288657,Exercise,2018,2018-12,2018-12-06,6,18,3
1,Readings and Review on Running Stride,2890761573,NaN,2158267779,2890761573,4288657,Exercise,2018,2018-12,2018-12-06,6,18,3
2,READ / Notes on Are Emotions of Natural Kind?,2939017612,NaN,178797715,2939017612,4288657,Studies: General,2018,2018-12,2018-12-06,6,18,3
3,Debug Site Down for GayRomLit,2939016120,NaN,179268527,2939016120,4288657,Misc Client Work,2018,2018-12,2018-12-06,6,18,3
4,Call with Jo Carol,2937620714,NaN,2182684133,2937620714,4288657,BookLoversCon,2018,2018-12,2018-12-06,6,0,3


In [41]:
# save to CSV
past_tasks.to_csv("data/todost-tasks-completed.csv", index=False)